In [43]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 18 18:30:23 2018

@author: Ryan
"""
#%%
import numpy as np
from sklearn import svm
import matplotlib.pyplot as plt
import cv2 as cv

import os
os.chdir("/Users/Ryan/Desktop/week10/Assignment 10T")
os.getcwd()

#%%
def separate(a, thresh_bright, loarea, uparea):
    
    #draw contours
    a_gray = cv.cvtColor(a, cv.COLOR_BGR2GRAY)
    ret, mask_a = cv.threshold(a_gray, thresh_bright, 255, 0)
    a2, contours, hierarchy = cv.findContours(mask_a, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(a, contours, -1, (0,0,255), 1)
    cv.namedWindow('a',cv.WINDOW_NORMAL)
    cv.imshow('a', a)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
        #cleaning and filtering the contours
    sub_image = []
    for contour in contours:
        #calculate the bounding rectangle
        #left = min(contour[:,0,0])#right = max(contour[:,0,0])#top = min(contour[:,0,1])#bottom = max(contour[:,0,1])
        x,y,w,h = cv.boundingRect(contour)
        #calculate the area of the bounding rectangle
        area = w*h
        
        #filter on the area, make a rectangle of the ok's, and put the corresponding sub-image in a list
        if area > loarea and area < uparea:
            cv.rectangle(a, (x, y), (x+w, y+h), (255,255,255), 2)
            sub_image.append(a[y:y+h, x:x+w])
            
    cv.namedWindow('img', cv.WINDOW_NORMAL)
    cv.imshow('img', a)
    cv.waitKey(0)
    cv.destroyAllWindows()
    
    sub_image = np.array(sub_image)
    
    for im in sub_image:
        cv.imshow('img', im)
        cv.waitKey(400)
        cv.destroyAllWindows()
    
    def processIm(im):
        im = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
        return cv.resize(im, (10,10), fx=0, fy=0, interpolation = cv.INTER_CUBIC)
    
    data = [processIm(im) for im in sub_image]
    data = np.array(data)
    return data

#%%
a = cv.imread('a.png')
data_a = separate(a, 200, 500, 2000)
target_a = np.zeros(len(data_a))
#%% for z
z = cv.imread('z.png')
data_z = separate(z, 150, 0, 3000)
target_z = np.zeros(len(data_z))

#%% for m
m = cv.imread('m.png')
data_m = separate(m, 150, 0, 3000)
target_m = np.ones(len(data_m))

#%% for y
y = cv.imread('y.png')
data_y = separate(y, 150, 1300, 3000)
target_y = np.ones(len(data_y)) + 1

data = np.vstack((data_z, data_m, data_y))
target = np.hstack((target_z, target_m, target_y))

#%% partition training and testing data


In [34]:
y.dtype

dtype('uint8')

In [77]:
def partition(data, target, p):
    l = len(data)
    print l
    n = int(l*p)
    print n
    train_i = np.random.choice(l, n, False)
    all_i = np.arange(l)
    all_i[train_i] = -1
    test_i = all_i[all_i > -1]

    train_data = np.reshape(data[train_i], (n, -1))
    train_target = target[train_i]
    test_data = np.reshape(data[test_i], (l-n, -1))
    test_target = target[test_i]
    
    return train_data, train_target, test_data, test_target


In [78]:
train_data, train_target, test_data, test_target = partition(data, target, 0.6)
from sklearn import svm
clf = svm.LinearSVC()
clf.fit(train_data, train_target)

44
26


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [82]:
predict_result = clf.predict(test_data)
print predict_result

print test_target
predict_result == test_target

[0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2.]
[0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2.]


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [83]:
accuracy = sum(predict_result==test_target)*1.0/(len(test_data))

In [84]:
accuracy

1.0

In [8]:
print data.shape
print target.shape

(44, 10, 10)
(44,)


In [46]:
a = cv.imread('a.png')

In [47]:
a.dtype

dtype('uint8')